In [ ]:
# 이 코랩 파일은 '학교 안내챗봇 만들기' 프로젝트 중 파인튜닝된 Gemma2b 모델을 gradio와 연결해서 챗봇 ui를 구성하는 부분입니다.
# 프로젝트 폴더 중 Test 폴더 안 여러 코랩 파일에서 개발 및 테스트를 거치고, 최종적으로 완성된 코드만 옮겨왔습니다.

In [ ]:
!pip install transformers
!pip install gradio
!pip install accelerate
!pip install torch
!pip install python-dotenv

In [ ]:
!huggingface-cli login # 허깅페이스 로그인
  # 개인 huggingface token을 입력

In [ ]:
# 6/7 테스트 성공 (영어, 한국어)

# import
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import gradio as gr

# 허깅페이스 API 토큰 지정
hf_token = '{huggingface-token}' # 이곳에 허깅페이스 토큰을 삽입

# 모델 지정 및 데이터타입 설정 (16bit)
model_id = "taeseo06/SchoolHelperChatbot-Gemma2B-Finetuning" # model: https://huggingface.co/datasets/taeseo06/SchoolHelperChatbot-dataset
dtype = torch.bfloat16

# 채팅 함수
def gemma_chat(message, history):
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        use_auth_token=hf_token,
        torch_dtype=dtype,
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    chat = [
        { "role": "user", "content": message },
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs, max_new_tokens=2048)

    response = tokenizer.decode(outputs[0])

    # 응답을 정리
    response_cleaned = re.split("model", response)

    # 응답 반환
    return response_cleaned[1]

# Gradio 인터페이스 생성 및 실행
gr.ChatInterface(gemma_chat).launch(debug=True)